# _***Simulacion de enfermedades cardiacas***_

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

## _***1: DATA WRANGLING.***_

Se carga el conjunto de datos que nos servira para hacer la simulacion de las enfermedades cardiacas, para cargar el conjunto de datos usamos el metodo `pd.read_csv` de la libreria _***Pandas***_.

In [2]:
url = "https://raw.githubusercontent.com/GabrielCastro1221/csv_dataScience/main/heart1.csv"
df = pd.read_csv(url)
df

age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0      52    1   0       125   212    0        1      168      0      1.0   
1      53    1   0       140   203    1        0      155      1      3.1   
2      70    1   0       145   174    0        1      125      1      2.6   
3      61    1   0       148   203    0        1      161      0      0.0   
4      62    0   0       138   294    1        1      106      0      1.9   
...   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
1020   59    1   1       140   221    0        1      164      1      0.0   
1021   60    1   0       125   258    0        0      141      1      2.8   
1022   47    1   0       110   275    0        0      118      1      1.0   
1023   50    0   0       110   254    0        0      159      0      0.0   
1024   54    1   0       120   188    0        1      113      0      1.4   

      slope  ca  thal  target  
0         2   2     3       0  
1         0   0     3       0  
2         0   0     3       0  
3         2   1     3       0  
4         1   3     2       0  
...     ...  ..   ...     ...  
1020      2   0     2       1  
1021      1   1     3       0  
1022      1   1     2       0  
1023      2   0     2       1  
1024      1   1     3       0  

[1025 rows x 14 columns]

A continuacion se creara una función que servira para crear una columna a partir de los datos alojados en la campo `target`.
Para desarrollar la funcion se hace lo siguiente:

+ Se crea una columna a partir de los datos alojados en la campo `target`. Esta columna contiene 2 valores "1" y "0" donde "1" es para las personas enfermas y "0" para las personas sanas. Para entender mejor estos datos se creara un campo de datos nuevo que se llamara `status` y este servira para analizar cada una de los datos de esta columna, donde aparece un "0" asignara la palabra sano y donde aparece un "1" asignara la palabra enfermo.

+ Se crea una variable con el nombre `result` la cual contendra el valor _***Enfermo***_ por defecto.

+ Se condiciona la fila que contiene los datos que se usaran para obtener los valores del campo `target` y validara que si el numero es igual a "0" a la variable se le reemplaza el valor "enfermo" por el valor "sano".

Una vez se obtiene el valor de la funcion, se inserta un campo de datos nuevo a la serie de datos que contendra la informacion que especificara si la persona esta sana o enferma. A este campo de datos se le asigna el nombre de `status`.

In [3]:
def sano_enfermo(fila):
    result = 'enfermo'
    if int(fila['target']) == 0:
        result = 'sano'
    return result
df['status'] = df.apply(sano_enfermo, axis=1)
df.head(20)

age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0    52    1   0       125   212    0        1      168      0      1.0   
1    53    1   0       140   203    1        0      155      1      3.1   
2    70    1   0       145   174    0        1      125      1      2.6   
3    61    1   0       148   203    0        1      161      0      0.0   
4    62    0   0       138   294    1        1      106      0      1.9   
5    58    0   0       100   248    0        0      122      0      1.0   
6    58    1   0       114   318    0        2      140      0      4.4   
7    55    1   0       160   289    0        0      145      1      0.8   
8    46    1   0       120   249    0        0      144      0      0.8   
9    54    1   0       122   286    0        0      116      1      3.2   
10   71    0   0       112   149    0        1      125      0      1.6   
11   43    0   0       132   341    1        0      136      1      3.0   
12   34    0   1       118   210    0        1      192      0      0.7   
13   51    1   0       140   298    0        1      122      1      4.2   
14   52    1   0       128   204    1        1      156      1      1.0   
15   34    0   1       118   210    0        1      192      0      0.7   
16   51    0   2       140   308    0        0      142      0      1.5   
17   54    1   0       124   266    0        0      109      1      2.2   
18   50    0   1       120   244    0        1      162      0      1.1   
19   58    1   2       140   211    1        0      165      0      0.0   

    slope  ca  thal  target   status  
0       2   2     3       0     sano  
1       0   0     3       0     sano  
2       0   0     3       0     sano  
3       2   1     3       0     sano  
4       1   3     2       0     sano  
5       1   0     2       1  enfermo  
6       0   3     1       0     sano  
7       1   1     3       0     sano  
8       2   0     3       0     sano  
9       1   2     2       0     sano  
10      1   0     2       1  enfermo  
11      1   0     3       0     sano  
12      2   0     2       1  enfermo  
13      1   3     3       0     sano  
14      1   0     0       0     sano  
15      2   0     2       1  enfermo  
16      2   1     2       1  enfermo  
17      1   1     3       0     sano  
18      2   0     2       1  enfermo  
19      2   0     2       1  enfermo

Se crea un diccionario al cual se le asigna el metodo `zip` para poder iterar su contenido, y poder asignar el numero "0" a las personas sanas y el numero "1" a las personas enfermas tomando los valores unicos de la columna `target` y asignandolos a los valores de la columna `status`.

In [4]:
resultado = dict(zip(df['target'].unique(), df['status'].unique()))
resultado

{0: 'sano', 1: 'enfermo'}

## _***2. Creacion del modelo KNN***_

Se crea la variable dependiente X y la variable independiente Y que serviran para hacer el entrenamiento del modelo.

+ _***X***_ contendra todos los registros con las caracteristicas de cada persona.

+ _***Y***_ contendra todos los datos que nos indica si la persona esta sana o enferma.

Se crean 4 variables las cuales serviran para hacer el entrenamiento y el testeo de los datos que contienen la variable dependiente e independiente. a las variables de prueba se le asigna el 20% de los datos y las variables de entrenamiento se le asigna el 80% de los datos.

In [5]:
X = df[['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']]
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.20)

Se crea el modelo K-NN y se le asigna el metodo `fit` que sirve para entrenar el modelo, a este metodo se le asignan la variable de entrenamiento X_train y la variable de entrenamiento Y_train.

In [6]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

Para saber que tan efectivo es el _***modelo K-NN***_ se crea la variable `estimate` a la cual se le asigna la metrica `score` de la libreria _***sklearn***_ que evalua el resultado de las variables de testeo `x_test` y `y_test` devolviendo la presicion del modelo, si el valor se acerca a "0" el modelo es malo, pero si el valor se acerca a "1" el modelo es bueno.

In [7]:
estimate = knn.score(X_test, y_test)
print(f'La efectividad del modelo KNN es de {estimate}')

La efectividad con el modelo de kNN es de 0.9121951219512195


Se puede observar que la acertividad de las variables de prueba del _***modelo K-NN***_ es muy buena ya que el valor se acerca demasiado a "1". El valor es "0.91" de acertividad.

Se aplicara el mismo metodo para evaluar las variables de entrenamiento x_train y y_train y asi poder ver que tan acertado es el resultado de la comparacion de dichas variables.

In [8]:
estimacion_entrenamiento = knn.score(X_train, y_train)
print(f'La efectividad de entrenamiento del modelo es {estimacion_entrenamiento}')

La efectividad de entrenamiento del modelo es 0.9902439024390244


Se puede observar que la acertividad de las variables de entrenamiento del _***modelo K-NN***_ es extremadamente buena ya que el valor se acerca demasiado a 1. El valor es 0.99 de acertividad.

In [9]:
df

age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0      52    1   0       125   212    0        1      168      0      1.0   
1      53    1   0       140   203    1        0      155      1      3.1   
2      70    1   0       145   174    0        1      125      1      2.6   
3      61    1   0       148   203    0        1      161      0      0.0   
4      62    0   0       138   294    1        1      106      0      1.9   
...   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
1020   59    1   1       140   221    0        1      164      1      0.0   
1021   60    1   0       125   258    0        0      141      1      2.8   
1022   47    1   0       110   275    0        0      118      1      1.0   
1023   50    0   0       110   254    0        0      159      0      0.0   
1024   54    1   0       120   188    0        1      113      0      1.4   

      slope  ca  thal  target   status  
0         2   2     3       0     sano  
1         0   0     3       0     sano  
2         0   0     3       0     sano  
3         2   1     3       0     sano  
4         1   3     2       0     sano  
...     ...  ..   ...     ...      ...  
1020      2   0     2       1  enfermo  
1021      1   1     3       0     sano  
1022      1   1     2       0     sano  
1023      2   0     2       1  enfermo  
1024      1   1     3       0     sano  

[1025 rows x 15 columns]

### _***3. Simulacion de la prediccion enfermadades cardiacas***_

Para hacer las predicciones correspondientes se usa el metodo `predict` de la libreria _***sklearn***_ y como parametro se le pasan cada uno de los datos que se encuentran en el conjunto de datos:

_***Para ingresar los datos de la persona guiese usando la siguiente tabla.***_

CAMPO            | DESCRIPCION
-----------------|------------------------------------------------------------------------
_***AGE***_      | Edad del paciente
_***SEX***_      | Sexo del paciente: 0 = Hombre - 1 = Mujer
_***CP***_       | Tipo de dolor en el pecho: 0 = angina tipica - 1 = angina atipica - 2 dolor no anginoso - 3 = asintomatico
_***TRESTBPS***_ | Presión arterial en reposo en mm Hg
_***CHOL***_     | Colesterol sérico en mg/dl
_***FBS***_      | Nivel de azúcar en la sangre en ayunas, clasificado como superior a 120 mg/dl: 1 = Verdadero - 0 = Falso
_***RESTECG***_  | Resultados electrocardiográficos en reposo: 0 = Normal - 1 = Tener anomalía de la onda ST-T - 2 = Mostrando hipertrofia ventricular izquierda probable o definitiva.
_***THALACH***_  | Frecuencia cardíaca máxima alcanzada durante una prueba de esfuerzo
_***EXANG***_    | Angina inducida por el ejercicio: 1 = Si - 0 = No
_***OLDPEAK***_  | Depresión del ST inducida por el ejercicio en relación con el reposo
_***SLOPE***_    | Pendiente del segmento ST del ejercicio máximo: 0 = pendiente ascendente - 1 = Plana - 2 = pendiente descendente
_***CA***_       | Número de vasos principales (0-4) coloreados por fluoroscopia
_***THAL***_     | Resultado de la prueba de esfuerzo con talio: 0 = Normal - 1 = Defecto solucionado - 2 = defecto reversible - 3 = No descrito
_***TARGET***_   | Estado de enfermedad cardíaca: 0 = Sano - 1 = Enfermo

In [10]:
predict = knn.predict([[47,1,0,110,275,0,0,118,1,1.0,1,1,2]])
resultado[predict[0]]

'sano'